<H1>N.B: This example is not currently working due to issues with recently updated dependencies</H1>

In [1]:
%matplotlib inline

In [4]:
import os
import netCDF4
import numpy as np
from geophys_utils import NetCDFGridUtils
from geophys_utils import NetCDFLineUtils
from geophys_utils import get_gdal_wcs_dataset, get_gdal_grid_values # N.B: GDAL WCS won't work on Alex's Windows laptop
import matplotlib.pyplot as plt

In [5]:
# Setup proxy as required
GA_STAFF_WIFI = False

if GA_STAFF_WIFI:
    os.environ['http_proxy'] = 'http://proxy.inno.lan:3128'
    os.environ['https_proxy'] = 'http://proxy.inno.lan:3128'

In [12]:
dem_wcs_url = 'http://services.ga.gov.au/site_9/services/DEM_SRTM_1Second_Hydro_Enforced/MapServer/WCSServer?service=WCS'

line_nc_path = "/iv65/Geoscience_Australia_Geophysics_Reference_Data_Collection/airborne_geophysics/SA/line/P1255/P1255-line-elevation-Marree.nc"
if not os.path.isfile(line_nc_path):
    line_nc_path = "https://dapds00.nci.org.au/thredds/dodsC/iv65/Geoscience_Australia_Geophysics_Reference_Data_Collection/airborne_geophysics/SA/line/P1255/P1255-line-elevation-Marree.nc"

grid_nc_path = "/iv65/Geoscience_Australia_Geophysics_Reference_Data_Collection/national_geophysical_compilations/Gravmap2019/Gravmap2019-grid-grv_cscba_1vd.nc"
if not os.path.isfile(grid_nc_path):
    grid_nc_path = "https://dapds00.nci.org.au/thredds/dodsC/iv65/Geoscience_Australia_Geophysics_Reference_Data_Collection/national_geophysical_compilations/Gravmap2019/Gravmap2019-grid-grv_cscba_1vd.nc"

bounds = (137, -29, 138, -28) # Bounding box within survey area

In [10]:
# Initialise NetCDFLineUtils object
# Code is at https://github.com/alex-ip/geophys_utils
#line_nc_dataset = netCDF4.Dataset(line_nc_path)
line_nc_dataset = netCDF4.Dataset(line_nc_path + '#fillmismatch') # Note work-around for bad _FillValue: https://github.com/Unidata/netcdf-c/issues/1299
netcdf_line_utils = NetCDFLineUtils(line_nc_dataset)

In [13]:
# Initialise NetCDFGridUtils object
# Code is at https://github.com/alex-ip/geophys_utils
grid_nc_dataset = netCDF4.Dataset(grid_nc_path)
netcdf_grid_utils = NetCDFGridUtils(grid_nc_dataset)

In [14]:
# Open GDAL dataset for DEM WCS
dem_gdal_dataset = get_gdal_wcs_dataset(dem_wcs_url)

ProxyError: HTTPConnectionPool(host='proxy.ga.gov.au', port=8080): Max retries exceeded with url: http://services.ga.gov.au/site_9/services/DEM_SRTM_1Second_Hydro_Enforced/MapServer/WCSServer?service=WCS&request=GetCapabilities&version=1.0.0 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002130C76F3C8>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')))

In [ ]:
# Get all coordinates and data points for line 100060
line_number, line_dict = next(netcdf_line_utils.get_lines(line_numbers=100060, variables='mag_awags'))
print(line_number, line_dict)

coordinates = line_dict['coordinates']
mag_awags = line_dict['mag_awags']

In [15]:
# Compute cumulative distances in metres for line 100060
utm_distances = netcdf_line_utils.coords2metres(coordinates)
print(utm_distances.shape, utm_distances)

NameError: name 'coordinates' is not defined

In [16]:
# Obtain values for sample points from gridded dataset
grid_values = np.array(netcdf_grid_utils.get_value_at_coords(coordinates, max_bytes=100))
print(grid_values.shape, grid_values)

NameError: name 'coordinates' is not defined

In [17]:
# Obtain DEM values via WCS
dem_values = get_gdal_grid_values(dem_gdal_dataset, coordinates, netcdf_line_utils.wkt)
print(dem_values.shape, dem_values)

NameError: name 'dem_gdal_dataset' is not defined

In [18]:
# Plot data points against line distance for line 100060
plt.figure(figsize=(30,20))   
plt.plot(utm_distances, line_dict['mag_awags'], 'b-', label='AWAGS-levelled magnetic values')        
plt.plot(utm_distances, grid_values, 'r-', label='IR gravity anomaly values')
plt.plot(utm_distances, dem_values, 'g-', label='1 Second Hydro Enforced DEM values')
plt.legend()

NameError: name 'utm_distances' is not defined

<Figure size 2160x1440 with 0 Axes>